<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка</a></span></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Требование: построить модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделать выводы.

**Описание данных**

 Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
!pip install imblearn

Импортируем необходимые для проекта библиотеки

In [80]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import re 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', None)
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

In [5]:
toxic_comments.head(20)

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,9,alignment on this subject and which are contrary to those of DuLithgow,0


Данные необходимо предобработать

### Предобработка

In [6]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [7]:
toxic_comments['toxic'].value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

Дисбаланс классов присутствует. Данный факт будет учтен при обучении моделей.

Удаляем лишний столбец `Unnamed: 0` из датафрейма

In [8]:
toxic_comments = toxic_comments.drop(['Unnamed: 0'], axis=1)

Создаем список **corpus** с текстами из столбца `text`. Это будет наш исходный набор данных для предобработки

In [9]:
corpus = list(toxic_comments['text'])

Необходимо очистить текст: привести к нижнему регистру, убрать лишние символы

In [10]:
def clear(text):
    text = re.sub(r"[^'a-zA-Z ]", ' ', text.lower()).split() 
    text = ' '.join(text)
    return text

Функция для получения части речи (POS tag) для лемматизации

In [11]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,
        'N': wordnet.NOUN,
        'V': wordnet.VERB,
        'R': wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

Функция для лемматизации текста: токенизирует текст , лемматизирует каждое слово с учетом его части речи, объединяет слова обратно в строку через пробел


In [12]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)])
    return lemmatized_text

Проведем очистку и лемматизацию текста из каждой строки датафрейма

In [13]:
tqdm.pandas()
toxic_comments['lemm_text'] = toxic_comments['text'].progress_apply(lambda x: lemmatize(clear(x)))

100%|██████████████████████████████████| 159292/159292 [04:08<00:00, 639.94it/s]


In [14]:
toxic_comments.head()

,text,toxic,lemm_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,explanation why the edits make under my username hardcore metallica fan be revert they be n't vandalism just closure on some gas after i vote at new york doll fac and please do n't remove the template from the talk page since i 'm retire now
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,d'aww he match this background colour i 'm seemingly stuck with thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,hey man i 'm really not try to edit war it 's just that this guy be constantly remove relevant information and talk to me through edits instead of my talk page he seem to care more about the format than the actual info
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,more i ca n't make any real suggestion on improvement i wonder if the section statistic should be later on or a subsection of type of accident i think the reference may need tidy so that they be all in the exact same format ie date format etc i can do that later on if no one else do first if you have any preference for format style on reference or want to do it yourself please let me know there appear to be a backlog on article for review so i guess there may be a delay until a reviewer turn up it 's list in the relevant form eg wikipedia good article nomination transport
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir be my hero any chance you remember what page that 's on


### Разделение данных на выборки

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    toxic_comments.drop(['text', 'toxic'], axis=1),
    toxic_comments['toxic'],
    test_size=0.25,
    stratify=toxic_comments['toxic'],
    random_state=12345
)

### TF-IDF

Загрузим необходимый модуль и стоп-слова для английского языка

In [39]:
nltk.download('stopwords') 
stop_words = list(set(stopwords.words('english')))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shanaaev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создадим корпус из лемматизированных слов для тренировочной выборки

In [40]:
lemm_corpus_train = X_train['lemm_text'].values
lemm_corpus_train.shape

(119469,)

In [41]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf_train = count_tf_idf.fit_transform(lemm_corpus_train)
tf_idf_train.shape

(119469, 128139)

Создадим корпус для тестовой выборки

In [42]:
lemm_corpus_test = X_test['lemm_text'].values
lemm_corpus_test.shape

(39823,)

In [43]:
tf_idf_test = count_tf_idf.transform(lemm_corpus_test)
tf_idf_test.shape

(39823, 128139)

**Вывод:** данные загружены и подготовлены для обучения модели. Лишний столбец удален, текст очищен от лишних символов, приведен к нижнему регистру и лемматизирован. Датасет разделен на обучающую и тестовую выборки в соотношении 75/25 с учетом баланса классов. Затем выполнено преобразование текста в числовой формат с помощью TF-IDF. Теперь можно переходить к обучению моделей.

## Обучение

### LogisticRegression

In [33]:
weight_for_class_0 = 1 / toxic_comments['toxic'].value_counts(normalize=True)[0]
weight_for_class_1 = 1 / toxic_comments['toxic'].value_counts(normalize=True)[1]

class_weights = {0: weight_for_class_0, 1: weight_for_class_1}

In [81]:
f1 = make_scorer(f1_score , average='macro')

In [82]:
pipe = Pipeline(
    [
       ('models', LogisticRegression(random_state=12345, class_weight='balanced'))
    ]
)

In [87]:
param_grid = [
    {
        'models': [LogisticRegression(random_state=12345, class_weight='balanced')],
        'models__C': range(1, 4),
    },
    {
        'models': [DecisionTreeClassifier(class_weight='balanced')],
        'models__max_depth': range(2, 5),
        'models__max_features': range(2, 5)
    },
    {
        'models': [
            CatBoostClassifier(
                verbose=False,
                iterations=200,
                class_weights=class_weights
            )
        ],
    }
]

In [88]:
grid = GridSearchCV(
    pipe, 
    param_grid, 
    cv=3,
    scoring=f1,
    n_jobs=-1,
    verbose=10
)
grid.fit(tf_idf_train, y_train)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 3/3; 2/13] START models=LogisticRegression(class_weight='balanced', random_state=12345), models__C=2
[CV 3/3; 2/13] END models=LogisticRegression(class_weight='balanced', random_state=12345), models__C=2;, score=0.860 total time=   1.4s
[CV 1/3; 8/13] START models=DecisionTreeClassifier(class_weight='balanced'), models__max_depth=3, models__max_features=3
[CV 1/3; 8/13] END models=DecisionTreeClassifier(class_weight='balanced'), models__max_depth=3, models__max_features=3;, score=0.473 total time=   0.2s
[CV 3/3; 10/13] START models=DecisionTreeClassifier(class_weight='balanced'), models__max_depth=4, models__max_features=2
[CV 3/3; 10/13] END models=DecisionTreeClassifier(class_weight='balanced'), models__max_depth=4, models__max_features=2;, score=0.474 total time=   0.1s
[CV 2/3; 13/13] START models=<catboost.core.CatBoostClassifier object at 0x1155bb810>
[CV 2/3; 13/13] END models=<catboost.core.CatBoostClassifier obj

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('models',
                                        LogisticRegression(class_weight='balanced',
                                                           random_state=12345))]),
             n_jobs=-1,
             param_grid=[{'models': [LogisticRegression(class_weight='balanced',
                                                        random_state=12345)],
                          'models__C': range(1, 4)},
                         {'models': [DecisionTreeClassifier(class_weight='balanced')],
                          'models__max_depth': range(2, 5),
                          'models__max_features': range(2, 5)},
                         {'models': [<catboost.core.CatBoostClassifier object at 0x303a7ea50>]}],
             scoring=make_scorer(f1_score, response_method='predict', average=macro),
             verbose=10)

In [89]:
grid.best_estimator_

Pipeline(steps=[('models',
                 LogisticRegression(C=3, class_weight='balanced',
                                    random_state=12345))])

In [90]:
grid.best_score_

0.8608878715654256

In [92]:
y_pred = grid.best_estimator_.predict(tf_idf_test)
f1_score(y_test, y_pred)

0.7556880936602607

**Вывод:**

На кросс-валидации лучшей моделью стала LogisticRegression с параметром C = 3. 
F1 на cv равен 0.86. На тестовой выборке - 0.75, что соответствует условию задачи

## Выводы

Разработан инструмент для автоматического обнаружения токсичных комментариев в интернет-магазине «Викишоп». 

Основные этапы проекта:  
1. **Загрузка и предобработка данных:**  
   - Удален лишний столбец;
   - Текст очищен от лишних символов, приведен к нижнему регистру и лемматизирован;  
   - Данные разделены на обучающую и тестовую выборки (75/25) с учетом баланса классов;  
   - Преобразование текста в числовой формат выполнено с помощью **TF-IDF**.  

2. **Обучение моделей:**  
Произведена кросс-валидация при разных гиперпараметрах из 3 основных моделей: 
   - **Logistic Regression**
   - **Decision Tree Classifier**
   - **CatBoost Classifier**
   
3. **Выбор наилучшей модели:**  
На кросс-валидации лучшей моделью стала LogisticRegression с параметром C = 3. 
F1 на cv равен 0.86. На тестовой выборке - 0.75, что соответствует условию задачи


Разработанный инструмент хорошо определяет токсичные комментарии и может быть использован для автоматической модерации пользовательского контента.